In [1]:
# Создание наборов данных
from sklearn.svm import SVC
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
import math

# Загрузка обучающего набора
data_train = pd.read_csv("dataset_train_yolo.csv")

# Загрузка тестового набора
data_test = pd.read_csv("dataset_test_yolo.csv")

In [2]:
# Вычисление угла между двумя прямыми
def angle(x0, y0, x1, y1, x2, y2):
    calc_angle = math.degrees(math.atan2(y2 - y0, x2 - x0) - math.atan2(y1 - y0, x1 - x0))
    if calc_angle < 0:
        calc_angle += 360
    return calc_angle

In [3]:
Angle_YOLO = ['RightElbowAngle', 'LeftElbowAngle', 'RightHandAngle', 'LeftHandAngle', 'RightKneeAngle', 'LeftKneeAngle',
                   'RightLegAngle', 'LeftLegAngle'
             ]
Key_Points_YOLO = ['Nose', 'LeftEye', 'RightEye', 'LeftEar', 'RightEar', 
                   'LeftShoulder', 'RightShoulder', 'LeftElbow', 'RightElbow', 
                   'LeftWrist', 'RightWrist', 'LeftHip', 'RightHip', 
                   'LeftKnee', 'RightKnee', 'LeftAnkle', 'RightAnkle'
                  ]

In [4]:
# Создание пустого датафрейма
def init_data(angle_name):
    data = []
    data.append("FileName")
    for p in angle_name:
        data.append(p)
    data.append("target")
    data = pd.DataFrame(columns = data) 
    return data

In [7]:
# Создание датасета каркасных моделей (набора углов и метки - названия позы)
def create_df_angle_pos(angle_name, df, name_file):
    data = init_data(angle_name)
    target = []
    for i in range(len(df)):
        temp = []
        temp += [df.loc[i]['FileName']]
        temp += [angle(df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'], 
                        df.loc[i]['RightWrist_x'], df.loc[i]['RightWrist_y'])]
        temp += [angle(df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'], 
                        df.loc[i]['LeftWrist_x'], df.loc[i]['LeftWrist_y'])] 
        temp += [angle(df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'],
                        df.loc[i]['RightElbow_x'], df.loc[i]['RightElbow_y'], 
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'])]
        temp += [angle(df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'],
                        df.loc[i]['LeftElbow_x'], df.loc[i]['LeftElbow_y'], 
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'])]
        temp += [angle(df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                        df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'], 
                        df.loc[i]['RightAnkle_x'], df.loc[i]['RightAnkle_y'])]
        temp += [angle(df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                        df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'], 
                        df.loc[i]['LeftAnkle_x'], df.loc[i]['LeftAnkle_y'])]
        temp += [angle(df.loc[i]['RightHip_x'], df.loc[i]['RightHip_y'],
                       df.loc[i]['RightKnee_x'], df.loc[i]['RightKnee_y'],
                       df.loc[i]['RightShoulder_x'], df.loc[i]['RightShoulder_y'])]
        temp += [angle(df.loc[i]['LeftHip_x'], df.loc[i]['LeftHip_y'],
                       df.loc[i]['LeftKnee_x'], df.loc[i]['LeftKnee_y'],
                       df.loc[i]['LeftShoulder_x'], df.loc[i]['LeftShoulder_y'])]
        temp += [df.loc[i]['target']]
        data.loc[i] = temp # Запись в датафрейм

    data.to_csv(name_file) # Запись датафрейма углов в файл
    return data

In [8]:
# Построение датафрейма угловых скелетных моделей обучающего набора
time_start = time.time()
data_tr_angle = create_df_angle_pos(Angle_YOLO, data_train, 'dataset_train_yolo_angle.csv')
time_end = time.time()
print('time: ', time_end-time_start)
print(data_tr_angle.info())
data_tr_angle.head()

time:  2.906088352203369
<class 'pandas.core.frame.DataFrame'>
Index: 1080 entries, 0 to 1079
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         1080 non-null   object 
 1   RightElbowAngle  1080 non-null   float64
 2   LeftElbowAngle   1080 non-null   float64
 3   RightHandAngle   1080 non-null   float64
 4   LeftHandAngle    1080 non-null   float64
 5   RightKneeAngle   1080 non-null   float64
 6   LeftKneeAngle    1080 non-null   float64
 7   RightLegAngle    1080 non-null   float64
 8   LeftLegAngle     1080 non-null   float64
 9   target           1080 non-null   object 
dtypes: float64(8), object(2)
memory usage: 92.8+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TRAIN/downdog/00000128.jpg,342.129240,276.810211,228.896501,331.665858,190.162943,179.789484,187.816537,145.514073,downdog
1,DATASET/TRAIN/downdog/00000129.jpg,314.581770,327.037972,94.317552,247.365465,165.440941,161.875275,66.059912,61.513083,downdog
2,DATASET/TRAIN/downdog/00000130.jpg,0.000000,0.000000,20.258953,20.908121,182.021294,182.348821,97.848886,96.882186,downdog
3,DATASET/TRAIN/downdog/00000131.jpg,189.138947,151.389954,143.959020,116.957221,189.665032,184.135467,339.161963,313.657414,downdog
4,DATASET/TRAIN/downdog/00000132.png,0.000000,0.000000,33.595478,34.364103,200.387035,179.748572,161.909927,123.446733,downdog


In [9]:
# Построение датафрейма угловых скелетных моделей тестового набора
time_start = time.time()
data_te_angle = create_df_angle_pos(Angle_YOLO, data_test, 'dataset_test_yolo_angle.csv')
time_end = time.time()
print('time: ', time_end-time_start)
print(data_te_angle.info())
data_te_angle.head()

time:  1.5003373622894287
<class 'pandas.core.frame.DataFrame'>
Index: 470 entries, 0 to 469
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   FileName         470 non-null    object 
 1   RightElbowAngle  470 non-null    float64
 2   LeftElbowAngle   470 non-null    float64
 3   RightHandAngle   470 non-null    float64
 4   LeftHandAngle    470 non-null    float64
 5   RightKneeAngle   470 non-null    float64
 6   LeftKneeAngle    470 non-null    float64
 7   RightLegAngle    470 non-null    float64
 8   LeftLegAngle     470 non-null    float64
 9   target           470 non-null    object 
dtypes: float64(8), object(2)
memory usage: 40.4+ KB
None


,FileName,RightElbowAngle,LeftElbowAngle,RightHandAngle,LeftHandAngle,RightKneeAngle,LeftKneeAngle,RightLegAngle,LeftLegAngle,target
0,DATASET/TEST/downdog/00000000.jpg,338.624574,338.189221,253.845263,272.272865,232.755539,144.003101,249.460126,145.496942,downdog
1,DATASET/TEST/downdog/00000001.JPG,197.997990,281.168449,184.989974,104.459494,156.100033,150.131203,287.261636,262.485779,downdog
2,DATASET/TEST/downdog/00000002.jpg,0.000000,0.000000,37.039138,38.995280,231.951914,136.059085,190.767107,68.089322,downdog
3,DATASET/TEST/downdog/00000003.jpg,0.000000,0.000000,0.000000,41.467525,241.504732,107.585444,311.766109,45.267594,downdog
4,DATASET/TEST/downdog/00000004.jpg,0.000000,0.000000,35.829724,33.327523,100.371229,263.147738,30.803267,226.775665,downdog
